In [1]:
import torch
import numpy as np
from load_data import *

In [49]:
train_data_dir = "/opt/ml/level2-klue-level2-nlp-03/baseline/dataset/train/train.csv"

In [50]:
train_dataset = load_data(train_data_dir)

In [56]:
train_dataset.head()

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation


In [143]:
from sklearn.model_selection import train_test_split
label = train_dataset['label']
train_test_split(train_dataset, test_size=0.2, shuffle=True, stratify=label, random_state=424)


[          id                                           sentence  \
 6667    6667  최창권(崔彰權, 천주교 세례명 바오로, 1934년 10월 26일 ~ 2008년 1월...   
 24635  24635  1991년부터 1995년까지 클리블랜드 인디언스의 미국 동부 및 라틴 아메리카 담당...   
 2934    2934  AFC 챔피언스리그 2011 일본 클럽 가시마 앤틀러스와의 원정경기에서 프로무대 첫...   
 17968  17968  곡성군이 지난 14일부터 17일까지 유근기 군수 주재로 부군수, 부서장, 부서별 전...   
 8739    8739  삼성전자는 한국에너지공단에서 시행하는 ‘으뜸효율 가전제품 구매비용 환급사업’에 맞춰...   
 ...      ...                                                ...   
 16230  16230  《테일즈런너》는 대한민국의 라온 엔터테인먼트가 개발, 2005년에 출시된 캐주얼 온...   
 25707  25707  현대 학자들은 그리스 정교회가 그리스인의 민족적 정체성을 지키고, 그리스 사회가 발...   
 21968  21968  김영록 전라남도지사는 “각종 취약시설에 대해 코호트 수준으로 관리를 강화하고, 필요...   
 27476  27476  5년동안의 극렬한 무력항쟁을 위해 의열단 지도부로써 거사를 기획하고, 단원확충과 교...   
 2422    2422                        김원효·심진화 부부는 지난 2011년에 결혼했다.   
 
       subject_entity object_entity                      label  
 6667           '최창권'         '崔彰權'        per:alternate_names  
 24635   '클리블랜드 인디언스'        '1995'                n

In [ ]:
from sklearn.model_selection import train_test_split
label = train_dataset['label']
train_test_split(train_dataset, test_size=0.2, shuffle=True, stratify=label, random_state=424)


In [4]:
# for itr, vals in train_dataset.iterrows():
#     if vals['label'] == 'org:alternate_names':
#         print(vals)

In [96]:
MODEL_NAME = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [99]:
concat_entity = []
for e01, e02 in zip(train_dataset['subject_entity'], train_dataset['object_entity']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)

tokenized_sentences = tokenizer(
      concat_entity,
      list(train_dataset['sentence']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=256,
      add_special_tokens=True,
      )

In [137]:
o = tokenizer(['tt'],['this'])
print(o)
tokenizer.decode(o['input_ids'][0])

{'input_ids': [[2, 87, 2013, 3, 25419, 3]], 'token_type_ids': [[0, 0, 0, 0, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}


'[CLS] tt [SEP] this [SEP]'

In [133]:
print(tokenizer.decode([87]))
print(tokenizer.decode([2013]))
tokenizer.decode([87,2013])

t
##t


'tt'

In [97]:
train_dataset['sentence'][0]

'〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [146]:
tokenized_sentences

{'input_ids': tensor([[    2,    11, 29830,  ...,     0,     0,     0],
        [    2,    11,  3772,  ...,     0,     0,     0],
        [    2,    11,  4104,  ...,     0,     0,     0],
        ...,
        [    2,    11, 18272,  ...,     0,     0,     0],
        [    2,    11, 15710,  ...,     0,     0,     0],
        [    2,    11, 15437,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [140]:
tokenizer.decode(tokenized_sentences['input_ids'][0][:30])

"[CLS]'비틀즈'[SEP]'조지 해리슨'[SEP] 〈 Something 〉 는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《"

In [86]:
def label_to_num(label):
    num_label = []
    with open('./dict_label_to_num.pkl', 'rb') as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label

In [101]:
train_label = label_to_num(train_dataset['label'].values)
RE_train_dataset = RE_Dataset(tokenized_sentences, train_label)

In [94]:
# # print(tokenized_sentences['input_ids'][0])
# print(tokenized_sentences['token_type_ids'][0])
# tokenizer.decode(tokenized_sentences['input_ids'][0])

In [93]:
# # tokenized sentence가 머지? 왜 다 첫번째 sentence 표기값은 없지..
# for i in range(len(tokenized_sentences)):
#     print(tokenized_sentences['token_type_ids'][i])
#     if i ==3:
#         break

In [104]:
model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [114]:
test = RE_train_dataset[0]
test.pop('labels')
for k,v in test.items():
    test[k]=v.unsqueeze(0)

In [116]:
out = model(**test)

In [120]:
out.pooler_output.shape

torch.Size([1, 768])

In [115]:
test

{'input_ids': tensor([[    2,    11, 29830,    11,     3,    11,  8373, 14113,  2234,    11,
              3,   168, 30985, 14451,  7088,  4586,   169,   793,  8373, 14113,
           2234,  2052,  1363,  2088, 29830,  2116, 14879,  2440,  6711,   170,
          21406, 26713,  2076, 25145,  5749,   171,  1421,   818,  2073,  4388,
           2062,    18,     3,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [107]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

inputs = tokenizer("NLP는 정말 재미있어", return_tensors="pt")
print(inputs)
# outputs = model(**inputs)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[   2,   50, 2237, 2325, 2259, 3944, 6001, 2051,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [92]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [78]:
cnt = 0
flag= False
for v in tokenized_sentences['token_type_ids'][0]:
    if v ==0 and not flag:
        cnt += 1
        flag = True
        break
    elif v!=0 and flag:
        cnt += 1
    elif v ==0 and flag:
        break
        
print(cnt)

1


In [75]:
print(tokenized_sentences['input_ids'][1])
print(tokenized_sentences['token_type_ids'][1])
tokenizer.decode(tokenized_sentences['input_ids'][1])

tensor([    2,    11,  3772,  2139,  2267,  2481,    11,     3,    11,  5605,
         2250,  2481,    11,     3,  6409,  2052,  4568,  2179,  6417,  2044,
         2315,  2481,   100,  5605,  2250,  2481,   100,  3772,  2139,  2267,
         2481,  2052, 16489,   711,  2170, 12827,  2097,  8646,  2481,    12,
        15283,    13,  3603,  1528,  2554,  2065,  4538,    18,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

"[CLS]'민주평화당'[SEP]'대안신당'[SEP] 호남이 기반인 바른미래당 · 대안신당 · 민주평화당이 우여곡절 끝에 합당해 민생당 ( 가칭 ) 으로 재탄생한다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [58]:
tokenized_sentences['input_ids'].shape

torch.Size([32470, 241])

# 튜토리얼 따라한 것

### Tokenizer

In [ ]:
# 궁금증
# pretrained를 가지고 오면
# tokenizer.tokenize 할 때 example의 tokenize 결과 나오는게 맞지?

# tokenize 따로 train 시키는 법!

* summary
    * training data의 언어를 이해할 수 있는 것 사용
    * 사용하는 pretrained model과 동일한 tokenizer인지 확인 -> vocab size mismath, special token이 unk로 처리될 수 있음
        * 단어의 개수와 special token이 완전히 일치하는 모델은 드물다! -> 각 모델에 맞는 것 꼭 불러오자
        * [Q] 그럼 finetuning할 때는?

In [5]:
# from ipywidgets import FloatProgress
from transformers import AutoTokenizer

example = "나는 학생이다."
# tokenization 결과를 보여줍니다.
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
# encoding이 어떻게 되는건지 모르겠다.
print('tokenization 결과 : ', tokenizer.tokenize(example))
print('tokenization + encoding 결과 : ', tokenizer.encode(example))

tokenization 결과 :  ['[UNK]', '[UNK]', '.']
tokenization + encoding 결과 :  [101, 100, 100, 119, 102]


In [7]:
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# print('tokenization 결과 : ', tokenizer.tokenize(example))
# print('tokenization + encoding 결과 : ', tokenizer.encode(example))
print(tokenizer.tokenize("[CLS]"))
print(tokenizer("[CLS]"))
tokenizer.add_special_tokens({'cls_token':'[CLS]'})
print(tokenizer.tokenize("[CLS]"))
print(tokenizer("[CLS]"))

['[CLS]']
{'input_ids': [2, 2, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
['[CLS]']
{'input_ids': [2, 2, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [4]:
from transformers import AutoTokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
print(tokenizer.tokenize("[CLS]"))
print(tokenizer("[CLS]"))
tokenizer.add_special_tokens({'cls_token':'[CLS]'})
print(tokenizer.tokenize("[CLS]"))
print(tokenizer("[CLS]"))

['[CLS]']
{'input_ids': [101, 101, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
['[CLS]']
{'input_ids': [101, 101, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


### Config

* summary
    * 사전 학습 모델이 가진 셋팅을 따르자
        * ㅁ델마다 vocab size, hidden dimension 등 파라미터 세팅이 다르다!
    * config update 방법
        * 호출 후 config attribute 으로 수정: `model_config.vocab_size = model_config.vocab_size+2`
        * 호출하면서 동시에 함수 아규먼트 값으로 넣어주는 방법 : `AutoConfig.from_pretrained(model_name, vocab_size=32002`
            * 이럴 경우 기존 config에 없는 값을 넣어주면 학습에 사용되지 않기 때문에 업데이트가 안되는 것 같다.
        * 커스텀 config 정의 후 업데이트 :
        ```
        custom_config = {'vocab_size':32000}
        model_config = AutoConfig.from_pretrained(model_name)
        model_config.update(custom_config)
        ```

In [9]:
from transformers import AutoConfig

model_name =  'klue/bert-base'
# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

In [15]:
model_config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

### pretrain model 불러오기
* summary
    * `from_config()` : config대로 모델 가져오는 것. pretrained weight 가져오는게 아님!
    * `from_pretrained()` : model config에 해당하는 모델과 pretrained weight 가져옴!
        * 스스로 학습한 모델은 model_name에 model이 저장된 directory 입력
    * 모델 가져오는 옵션
        * 기본 모델 : hidden state 출력되는 기본 모델
        * down stream task 모델 : 일반적인 task를 쉽게 수행할 수 있게 미리 기본 모델 + head 설정 모델
            * output은 task에 적합한 dimension으로 미리 저장돼있음

In [16]:
from transformers import AutoConfig, AutoModelForQuestionAnswering
model_name =  'klue/bert-base'

# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

# 모델을 정의합니다.

# option 1 : config에서 정의한 모델을 가져오기 (initial)
# model = AutoModelForQuestionAnswering.from_config(config)

# option 2 : config에서 정의한 사전학습된 모델을 가져오기 (pretrained)
model = AutoModelForQuestionAnswering.from_pretrained(
        model_name, config=model_config
    )
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

### 전체 학습시 선언 필요한 것
* model_name 선언
* model이 사용한 tokenizer
* model이 사용한 config
* model 불러오기!

In [17]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(
    model_name,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

# 모델을 학습해보자
     * [참고](https://github.com/yukyunglee/transformers-resources/blob/master/tutorial/02%20advanced%20tutorial.ipynb)

In [18]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

inputs = tokenizer("NLP는 정말 재미있어", return_tensors="pt")
outputs = model(**inputs)

cls_output = outputs.pooler_output

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
outputs.last_hidden_state.shape

torch.Size([1, 9, 768])

In [21]:
outputs.pooler_output.shape

torch.Size([1, 768])

In [39]:
inputs['input_ids']

tensor([[   2,   50, 2237, 2325, 2259, 3944, 6001, 2051,    3]])

In [38]:
tokenizer.decode(inputs['input_ids'][0].tolist())

'[CLS] NLP는 정말 재미있어 [SEP]'

In [28]:
tokenizer.encode("NLP는 정말 재미있어")

[2, 50, 2237, 2325, 2259, 3944, 6001, 2051, 3]

In [29]:
tokenizer.decode([2, 50, 2237, 2325, 2259, 3944, 6001, 2051, 3])

'[CLS] NLP는 정말 재미있어 [SEP]'

In [34]:
# 어떻게 알아서 떼내어주는거지..
tokenizer.decode([3944,6001, 2051])

'정말 재미있어'

# Tokenizer 궁금한 상황 추가
* vocab을 추가할 때 vocab의 갯수(즉 vocab의 길이) 와 차원 추가 여부 

In [ ]:
# tokenizer 훈련 부분 : https://colab.research.google.com/drive/1TWyT0wTv0Z0hVjLDMaG4qcNZZtdCgGux#scrollTo=VfEKVuu6GnW7
# 

In [46]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

# # 모델을 불러오기전에 vocab을 수정하면 pretrained config와 충돌이 일어나 에러가 발생하니 주의
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
)

# before add
print('before adding')
print(model.get_input_embeddings())
print(len(tokenizer))

# special token 추가하기 
special_tokens_dict = {'additional_special_tokens': ['[special1]','[special2]','[special3]','[special4]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# token 추가하기
new_tokens = ['COVID', 'hospitalization']
num_added_toks = tokenizer.add_tokens(new_tokens)

# # 기존 config로 모델을 불러오기
# # 모델을 불러오기전에 vocab을 수정하면 pretrained config와 충돌이 일어나 에러가 발생하니 주의
# model = AutoModelForQuestionAnswering.from_pretrained(
#     model_name,
#     config=config,
# )
print('after adding')
print(model.get_input_embeddings())
# tokenizer config 수정해주기 (추후에 발생할 에러를 줄이기 위해)
config.vocab_size = len(tokenizer)

print('after resizing')

# model의 token embedding 사이즈 수정하기
model.resize_token_embeddings(len(tokenizer))
print(model.get_input_embeddings())


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

before adding
Embedding(32000, 768, padding_idx=0)
32000
after adding
Embedding(32000, 768, padding_idx=0)
after resizing
Embedding(32006, 768)
